# Structural patterns

A Decorator is a pattern that allows you to add new functionality to objects by wrapping them around.

Fortunately for Python developers, this pattern is built in by default.

More details about working with decorators can be found in the "Functions" module. Here we will once again give an example of working with decorators in Python.

In [1]:
def func_time_decorator(func):
    import time
    def wrapper(*args, **kwargs):
        start = time.time()
        return_value = func(*args, **kwargs)
        end = time.time()
        print('[*] Время выполнения: {} секунд.'.format(end-start))
        return return_value
    return wrapper

Facade is a template that provides an interface to a complex system of classes, third-party libraries, and frameworks.

Example: connecting the new functionality of the Prometeus (for monitoring), Jaeger (for tracing) libraries is conveniently done using classes that isolate the call and work with this library, separating it from the classes of the system being developed.

In [2]:
class FileAdapterFacade:
     """
         Facade for file access that hides
         the specific system where the files are located
     """
     def __init__(self, subsystem) -> None:
         self._file_system = subsystem
     def get_file(self, file_name):
         return self._file_system.get(file_name)

class LocalStorager:
     """
        Accessing files on disk
     """
     def get_file(self, file_name):
         pass

class CephStorager:
     """
         Access to files that are located remotely in the ceph bucket
         This class can also implement
         working with the library, connecting to ceph
     """
     def get_file(self, file_name):
         connector = self.connect()
         connector.set_s3_mode()
         connector.turn_off_ssl()
         connector.establish()
         file = connector.get_file(file_name)
         return file

An adapter is a pattern that allows objects with different interfaces to work together.

Example: often when developing several systems, it is necessary to develop an adapter system that connects several systems through it into a single one. Within one project, this can be a class that prepares data, and separate classes adapt it for sending via rest or kafka.

In [ ]:
# coding=utf-8
import requests

CORRELATION_ID = "correlationId" 

class HttpRequest:
    def __init__(self, url):
        self.url = url

    def request(self, obj):
        try:
            if CORRELATION_ID not in obj.headers.keys():
                raise AttributeError
            else:
                response = requests.get(
                    self.url,
                    params=obj.params,
                    headers=obj.headers,
                )
                print(response)
        except AttributeError:
            print("not correlationId in headers")

class OldClass:
    def __init__(self, headers, params):
        self.headers = headers
        self.params = params

class OldClassHttpRequestAdapter:
    def __init__(self, obj):
        self.obj = obj
    @property
    def headers(self):
        if CORRELATION_ID not in self.obj.headers.keys():
            self.obj.headers[CORRELATION_ID] = "1232-1112-3333"
        return self.obj.headers
    @property
    def params(self):
        return self.obj.params

http_request = HttpRequest("https://github.com")
old_class_obj = OldClass({"test":"111"}, {"params1":"value1"})
adapter_obj = OldClassHttpRequestAdapter(old_class_obj)
http_request.request(old_class_obj) 
http_request.request(adapter_obj)

A Composite is a template that allows you to group multiple objects into a tree and then work with it as if it were a single object.

Example: when developing a program for Windows / Mac with a GUI, the graphical interface can be represented as a tree, which can be assembled into a single whole using a linker.

In [ ]:
from abc import ABC, abstractmethod

class GUIAbstractComponent(ABC):
     """
     The base class is an abstract GUI interface component.
     """
     @property
     def parent(self):
         return self._parent
     @parent.setter
     def parent(self, parent):
         self._parent = parent
     def add(self, component):
         pass
     def remove(self, component):
         pass
     @abstractmethod
     def action(self):
         pass

class Button(GUIAbstractComponent):
     """
         Has no nested components, performs actions.
     """
     def action(self):
         return "Button"

class EditBox(GUIAbstractComponent):
    def action(self):
        return "Leaf"

class Window(GUIAbstractComponent):
     """
         A container that contains simpler objects
     """
     def __init__(self):
         self._children = []
     def add(self, component):
         self._children.append(component)
         component.parent = self
     def remove(self, component):
         self._children.remove(component)
         component.parent = none
     def is_composite(self):
         return True
     def action(self):
         """
             The container walks over its children and causes them to
             actions, summarizes and gives the result.
             Thus, it is possible to organize redrawing
             all components in the GUI.
         """
         results = []
         for child in self._children:
             results.append(child.action())
         return f"Branch({'+'.join(results)})"

if __name__ == "__main__":
     window = Window()
     window.add(Button())
     window.add(EditBox())

Proxy is a template that allows you to pass "proxies" instead of real objects, which allow you to intercept the call and perform an action before and after the call.

Example: you are using a library that has implemented access to some resource on your network, but it does not check for the availability of this resource. Instead of rewriting your code, you can replace the class object from the library with your own proxy class that implements the same method, but first checks the availability of the resource on the network on its own.

In [4]:
class LocalWeb():
     def request(self):
         print("request to localweb")

class Proxy(LocalWeb):
     """
         Inherited from a class from the library
     """
     def __init__(self, local_web):
         self._local_web = local_web
     def request(self):
         if self.check_access():
             self._local_web.request()
     def check_access(self) -> bool:
         return True